In [1]:
#! python2
# -*- coding: utf-8 -*-
import pprint
import pymongo
import datetime
import numpy as np
from pymongo import MongoClient

from scipy import spatial

import pylab

import os

import progressbar

db_name = 'twitter'
col_name = 'cluster_score'

In [2]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

from igraph import *
import igraph
import math
from operator import itemgetter



In [3]:
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('thai'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)

def go_print( input ):
    MyPrettyPrinter().pprint(input)
    # ppp = pprint.PrettyPrinter(indent=4)
    # ppp.pprint(input)
    return;

def get_midnight(time):
    return time.replace(minute=0, hour=0, second=0, microsecond=0)

def get_time_gap(time,hour_gap=1,min_gap=1):
    h = time.hour
    m = time.minute
    o_h = h/hour_gap*hour_gap
    o_m = m/min_gap*min_gap
    return time.replace(hour=o_h, minute=o_m, second=0, microsecond=0)

def get_week_year(time):
    return tuple([time.isocalendar()[0], time.isocalendar()[1]])

def get_thai_midnight(time):
    out = time + datetime.timedelta(hours=7)
    out = out.replace(minute=0, hour=0, second=0, microsecond=0) - datetime.timedelta(hours=7)
    return out

In [4]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[col_name].find({})

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    all_score = {}
    d_all_score = {}
    for doc in cursor:
        data += 1
        bar.update(data)
        description = doc['description']
        levels = doc['levels']
        score_type = doc['type']
        
        completeness_score = doc['completeness_score']
        homogeneity_score = doc['homogeneity_score']
        v_measure_score = doc['v_measure_score']
        accuracy = doc['accuracy']

        if not all_score.has_key(description):
            all_score[description] = {}
            d_all_score[description] = {}
            d_all_score[description]['completeness_score'] = []
            d_all_score[description]['homogeneity_score'] = []
            d_all_score[description]['v_measure_score'] = []
            d_all_score[description]['accuracy'] = []
        
        if not all_score[description].has_key(levels):
            all_score[description][levels] = {}
            
        if not all_score[description][levels].has_key(score_type):
            all_score[description][levels][score_type] = {}
            all_score[description][levels][score_type]['completeness_score'] = []
            all_score[description][levels][score_type]['homogeneity_score'] = []
            all_score[description][levels][score_type]['v_measure_score'] = []
            all_score[description][levels][score_type]['accuracy'] = []

            
        all_score[description][levels][score_type]['completeness_score'].append(completeness_score)
        all_score[description][levels][score_type]['homogeneity_score'].append(homogeneity_score)
        all_score[description][levels][score_type]['v_measure_score'].append(v_measure_score)
        all_score[description][levels][score_type]['accuracy'].append(accuracy)
        
        d_all_score[description]['completeness_score'].append(completeness_score)
        d_all_score[description]['homogeneity_score'].append(homogeneity_score)
        d_all_score[description]['v_measure_score'].append(v_measure_score)
        d_all_score[description]['accuracy'].append(accuracy)
        
          
    bar.finish()

[########################################################################] 100%


In [5]:
for description,v1 in all_score.items():
    for levels,v2 in v1.items():
        for score_type,v3 in v2.items():
            for score_name, score_list in v3.items():
                print description,levels,score_type,score_name, 1.0*sum(score_list)/len(score_list)
            print ''

Affinity Propagation topic 1 homogeneity_score 0.525894089329
Affinity Propagation topic 1 v_measure_score 0.642072433225
Affinity Propagation topic 1 completeness_score 0.845784570649
Affinity Propagation topic 1 accuracy 0.846572772796

Affinity Propagation topic 2 homogeneity_score 0.525950425975
Affinity Propagation topic 2 v_measure_score 0.641871726409
Affinity Propagation topic 2 completeness_score 0.844958069993
Affinity Propagation topic 2 accuracy 0.847670079471

Affinity Propagation topic 4 homogeneity_score 0.540378466839
Affinity Propagation topic 4 v_measure_score 0.621258133287
Affinity Propagation topic 4 completeness_score 0.75100432104
Affinity Propagation topic 4 accuracy 0.839902015422

Affinity Propagation major_class 1 homogeneity_score 0.648051898117
Affinity Propagation major_class 1 v_measure_score 0.372612651433
Affinity Propagation major_class 1 completeness_score 0.270698417252
Affinity Propagation major_class 1 accuracy 0.61086233679

Affinity Propagation m

In [6]:
for description,v1 in d_all_score.items():
    for score_name, score_list in v1.items():
        print description,score_name, 1.0*sum(score_list)/len(score_list)
    print ''

Affinity Propagation homogeneity_score 0.617896089104
Affinity Propagation v_measure_score 0.550282883052
Affinity Propagation completeness_score 0.572911417603
Affinity Propagation accuracy 0.767435462261

DBSCAN homogeneity_score 0.845530674189
DBSCAN v_measure_score 0.652505385513
DBSCAN completeness_score 0.597916870732
DBSCAN accuracy 0.822384722029

Markov Cluster homogeneity_score 0.74533033957
Markov Cluster v_measure_score 0.655571374883
Markov Cluster completeness_score 0.66723515555
Markov Cluster accuracy 0.842201325828

